In [3]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA
from quant_utils.performance import Performance

In [85]:
query_sql = """
SELECT
	a.END_DATE,
	a.INDEX_CODE,
	a.LEVEL_NUM,
	a.TYPE_NAME,
	a.RETURN_RATE / 100 AS RETURN_RATE 
FROM
	fund_derivatives_inner_fund_ret a

WHERE
	1 = 1 
	AND a.LEVEL_NUM = 3 
	AND a.TYPE_NAME IN (
'主动权益-A股主题-无主题',
'主动权益-港股主题-无主题',
'固收-中长债-信用债',
'固收-中长债-利率债',
'固收-中长债-券种均衡',
'固收-短债-中短债',
'固收-短债-短债',
'固收-短债-超短债'
) 
and END_DATE >= '20211231'
"""
ret_df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [87]:
ret_df = ret_df.pivot_table(index="END_DATE", columns="TYPE_NAME", values="RETURN_RATE")
ret_df = ret_df.dropna(axis=0, how="all")

In [88]:
ret_df.corr()

TYPE_NAME,主动权益-A股主题-无主题,主动权益-港股主题-无主题,固收-中长债-信用债,固收-中长债-利率债,固收-中长债-券种均衡,固收-短债-中短债,固收-短债-短债,固收-短债-超短债
TYPE_NAME,,,,,,,,
主动权益-A股主题-无主题,1.000000,0.706871,-0.057749,-0.085354,-0.077553,-0.057729,-0.072928,-0.046584
主动权益-港股主题-无主题,0.706871,1.000000,-0.147588,-0.153978,-0.157936,-0.136451,-0.142918,-0.117557
固收-中长债-信用债,-0.057749,-0.147588,1.000000,0.830578,0.932029,0.966751,0.919928,0.845024
固收-中长债-利率债,-0.085354,-0.153978,0.830578,1.000000,0.967042,0.769150,0.722035,0.582011
固收-中长债-券种均衡,-0.077553,-0.157936,0.932029,0.967042,1.000000,0.875904,0.827477,0.703717
固收-短债-中短债,-0.057729,-0.136451,0.966751,0.769150,0.875904,1.000000,0.972417,0.921797
固收-短债-短债,-0.072928,-0.142918,0.919928,0.722035,0.827477,0.972417,1.000000,0.946031
固收-短债-超短债,-0.046584,-0.117557,0.845024,0.582011,0.703717,0.921797,0.946031,1.000000


In [80]:
ret_df["固收-短债-短债"].dropna()

END_DATE
2012-11-02    0.000000
2012-11-09    0.000988
2012-11-16    0.000987
2012-11-23    0.000986
2012-11-30    0.000000
                ...   
2023-07-31    0.000088
2023-08-01    0.000096
2023-08-02    0.000160
2023-08-03    0.000188
2023-08-04    0.000183
Name: 固收-短债-短债, Length: 2571, dtype: float64

计算风格相关性

In [90]:
query_sql = """
SELECT
	a.TRADE_DT,
	b.S_INFO_NAME,
	a.S_DQ_PCTCHANGE / 100 as RETURN_RATE
FROM
	aindex_eod_prices a 
	join aindex_description b on a.S_INFO_WINDCODE = b.S_INFO_WINDCODE
WHERE
	1 = 1 
	AND b.TICKER_SYMBOL IN (
		'399372',
		'399373',
		'399374',
		'399375',
	'399376',
	'399377') 
    and TRADE_DT >= '20211231'
    UNION SELECT
	a.END_DATE as TRADE_DT,
	a.TYPE_NAME as S_INFO_NAME,
	a.RETURN_RATE / 100 AS RETURN_RATE 
FROM
	fund_derivatives_inner_fund_ret a
WHERE
	1 = 1 
	AND a.LEVEL_NUM = 3 
	AND a.TYPE_NAME IN ( 
'固收-中长债-信用债',
'固收-中长债-利率债',
'固收-短债-中短债',
'固收-短债-短债',
'固收-短债-超短债'
)
and END_DATE >= '20211231'
"""
ret_df = DB_CONN_JJTG_DATA.exec_query(query_sql)
ret_df = ret_df.pivot_table(index="TRADE_DT", columns='S_INFO_NAME', values="RETURN_RATE")

In [92]:
ret_df.corr()

S_INFO_NAME,中盘价值,中盘成长,固收-中长债-信用债,固收-中长债-利率债,固收-短债-中短债,固收-短债-短债,固收-短债-超短债,大盘价值,大盘成长,小盘价值,小盘成长
S_INFO_NAME,,,,,,,,,,,
中盘价值,1.000000,0.736492,-0.068401,-0.098719,-0.059040,-0.068596,-0.029403,0.795808,0.668842,0.945200,0.798399
中盘成长,0.736492,1.000000,-0.076599,-0.097113,-0.070439,-0.078481,-0.051405,0.442216,0.910578,0.731300,0.926249
固收-中长债-信用债,-0.068401,-0.076599,1.000000,0.830578,0.966751,0.919928,0.845024,-0.110561,-0.141406,-0.034126,-0.023602
固收-中长债-利率债,-0.098719,-0.097113,0.830578,1.000000,0.769150,0.722035,0.582011,-0.137773,-0.156336,-0.059615,-0.058806
固收-短债-中短债,-0.059040,-0.070439,0.966751,0.769150,1.000000,0.972417,0.921797,-0.094275,-0.136253,-0.026808,-0.021841
固收-短债-短债,-0.068596,-0.078481,0.919928,0.722035,0.972417,1.000000,0.946031,-0.104878,-0.152563,-0.036991,-0.029002
固收-短债-超短债,-0.029403,-0.051405,0.845024,0.582011,0.921797,0.946031,1.000000,-0.056506,-0.115412,-0.007773,-0.006540
大盘价值,0.795808,0.442216,-0.110561,-0.137773,-0.094275,-0.104878,-0.056506,1.000000,0.503211,0.724402,0.453774
大盘成长,0.668842,0.910578,-0.141406,-0.156336,-0.136253,-0.152563,-0.115412,0.503211,1.000000,0.639552,0.802519


风格及券种久期

In [93]:
query_sql = """
SELECT
	a.TRADE_DT,
	b.S_INFO_NAME,
	a.S_DQ_PCTCHANGE / 100 AS RETURN_RATE 
FROM
	aindex_eod_prices a
	JOIN aindex_description b ON a.S_INFO_WINDCODE = b.S_INFO_WINDCODE 
WHERE
	1 = 1 
	AND b.TICKER_SYMBOL IN ( '399372', '399373', '399374', '399375', '399376', '399377' ) 
    and TRADE_DT >= '20220101'
    UNION
SELECT
	a.TRADE_DT,
	a.SEC_SHORT_NAME AS S_INFO_NAME,
	a.S_DQ_PCTCHANGE / 100 AS RETURN_RATE 
FROM
	bond_chinabondindexquote a 
WHERE
	1 = 1 
	AND a.TICKER_SYMBOL IN (
		'CBA05821',
		'CBA05831',
		'CBA05841',
		'CBA05851',
		'CBA05861',
		'CBA03511',
		'CBA03521',
	'CBA03531' 
	)
    and TRADE_DT >= '20220101'
"""
ret_df = DB_CONN_JJTG_DATA.exec_query(query_sql)
ret_df = ret_df.pivot_table(index="TRADE_DT", columns='S_INFO_NAME', values="RETURN_RATE")

In [94]:
ret_df.dropna().corr()

S_INFO_NAME,中债-国债及政策性银行债指数_1-3年,中债-国债及政策性银行债指数_10年以上,中债-国债及政策性银行债指数_3-5年,中债-国债及政策性银行债指数_5-7年,中债-国债及政策性银行债指数_7-10年,中债-高信用等级企业债指数_1-3年,中债-高信用等级企业债指数_1年以下,中债-高信用等级企业债指数_3-5年,中盘价值,中盘成长,大盘价值,大盘成长,小盘价值,小盘成长
S_INFO_NAME,,,,,,,,,,,,,,
中债-国债及政策性银行债指数_1-3年,1.000000,0.687944,0.908314,0.834284,0.776454,0.683468,0.570804,0.699997,-0.130210,-0.112208,-0.167616,-0.169945,-0.088222,-0.069985
中债-国债及政策性银行债指数_10年以上,0.687944,1.000000,0.830112,0.865944,0.892940,0.433575,0.283410,0.499966,-0.082096,-0.057289,-0.171443,-0.117768,-0.040525,-0.020140
中债-国债及政策性银行债指数_3-5年,0.908314,0.830112,1.000000,0.953198,0.925519,0.532939,0.381209,0.580692,-0.102042,-0.074279,-0.164457,-0.124042,-0.066149,-0.038266
中债-国债及政策性银行债指数_5-7年,0.834284,0.865944,0.953198,1.000000,0.952532,0.495138,0.353052,0.548905,-0.072027,-0.061735,-0.141690,-0.119917,-0.036798,-0.021374
中债-国债及政策性银行债指数_7-10年,0.776454,0.892940,0.925519,0.952532,1.000000,0.433921,0.293784,0.491921,-0.087406,-0.049855,-0.172633,-0.111931,-0.051265,-0.015523
中债-高信用等级企业债指数_1-3年,0.683468,0.433575,0.532939,0.495138,0.433921,1.000000,0.851275,0.966393,-0.041103,-0.050690,-0.077788,-0.113594,-0.020931,0.000787
中债-高信用等级企业债指数_1年以下,0.570804,0.283410,0.381209,0.353052,0.293784,0.851275,1.000000,0.808427,0.014607,0.014414,-0.032302,-0.076843,0.037466,0.068406
中债-高信用等级企业债指数_3-5年,0.699997,0.499966,0.580692,0.548905,0.491921,0.966393,0.808427,1.000000,-0.038144,-0.049077,-0.091329,-0.115784,-0.012347,0.008729
中盘价值,-0.130210,-0.082096,-0.102042,-0.072027,-0.087406,-0.041103,0.014607,-0.038144,1.000000,0.736279,0.795790,0.669208,0.945157,0.798241


自动挡VS拼盘

In [181]:
query_sql = """
SELECT
	a.END_DATE as TRADE_DT,
	a.TYPE_NAME as S_INFO_NAME,
	a.RETURN_RATE / 100 AS RETURN_RATE 
FROM
	fund_derivatives_inner_fund_ret a
WHERE
	1 = 1 
	AND a.LEVEL_NUM = 3 
	AND a.TYPE_NAME IN ( 
'固收-中长债-信用债',
'固收-中长债-利率债',
'固收-短债-短债',
"主动权益-A股主题-无主题"
)
and END_DATE >= '20150101'
union 
SELECT
	a.END_DATE as TRADE_DT,
	a.TYPE_NAME as S_INFO_NAME,
	a.RETURN_RATE / 100 AS RETURN_RATE 
FROM
	fund_derivatives_inner_fund_ret a
WHERE
	1 = 1 
	AND a.LEVEL_NUM = 2 
	AND a.TYPE_NAME IN ( 
    "固收+-(05-15)",
    "固收+-(15-25)"
)
and END_DATE >= '20150101'
"""
ret_df = DB_CONN_JJTG_DATA.exec_query(query_sql)
ret_df = ret_df.pivot_table(index="TRADE_DT", columns='S_INFO_NAME', values="RETURN_RATE")
ret_df = ret_df.dropna()

In [182]:
ret_df["拼盘05-15"] = ret_df["主动权益-A股主题-无主题"]*0.1 + ret_df["固收-中长债-信用债"]*0.9
ret_df["拼盘15-25"] = ret_df["主动权益-A股主题-无主题"]*0.2 + ret_df["固收-中长债-信用债"]*0.8
ret_df["多策略05-15"] = ret_df["拼盘05-15"]*0.7 + ret_df["固收+-(05-15)"]*0.3
ret_df["多策略15-25"] = ret_df["拼盘15-25"]*0.7 + ret_df["固收+-(15-25)"]*0.3

In [175]:
ret_df = (ret_df + 1).cumprod()

In [179]:
ret_df.to_excel("D:/拼盘.xlsx")

In [176]:
result_list = []
for col in ret_df.columns:
    temp = Performance(ret_df[col]).stats().T
    temp.index = [col]
    result_list.append(temp)
result = pd.concat(result_list)

In [177]:
result

,起始日期,结束日期,累计收益率,年化收益率,年化波动率,收益波动比,最大回撤,年化收益回撤比,最大回撤日,最大回撤修复
主动权益-A股主题-无主题,2015-01-05,2023-08-07,0.544449,0.051893,0.226284,0.229326,0.488812,0.106161,20190103,580
固收+-(05-15),2015-01-05,2023-08-07,0.398215,0.039785,0.022392,1.776735,0.038468,1.034222,20150708,161
固收+-(15-25),2015-01-05,2023-08-07,0.500258,0.048344,0.042181,1.146108,0.078283,0.617564,20150708,737
固收-中长债-信用债,2015-01-05,2023-08-07,0.380324,0.038227,0.007282,5.249594,0.026068,1.466453,20161220,412
固收-中长债-利率债,2015-01-05,2023-08-07,0.237105,0.025074,0.011616,2.158576,0.026147,0.958981,20200713,303
固收-短债-短债,2015-01-05,2023-08-07,0.29718,0.030747,0.004456,6.899688,0.013326,2.307241,20161220,149
拼盘05-15,2015-01-05,2023-08-07,0.4231,0.041922,0.023539,1.780933,0.046537,0.900819,20150708,126
拼盘15-25,2015-01-05,2023-08-07,0.460968,0.045111,0.045613,0.989011,0.097726,0.461611,20150915,966
多策略05-15,2015-01-05,2023-08-07,0.410846,0.040874,0.022049,1.853735,0.042505,0.961628,20150708,152
多策略15-25,2015-01-05,2023-08-07,0.481026,0.046771,0.042913,1.089915,0.08725,0.536064,20150708,824


In [133]:
query_sql = """
SELECT
	a.REPORT_DATE,
	a.LEVEL_1,
	a.LEVEL_2,
	a.LEVEL_3,
	a.leverage,
	b.DURATION,
	c.CASH_RATIO_IN_NA,
	c.EQUITY_RATIO_IN_NA,
	c.BOND_RATIO_IN_NA,
	c.F_PRT_COVERTBONDTONAV
FROM
	`chentiancheng`.`view_fund_derivatives_bond_type` a
	JOIN fund_style_bond b ON b.TICKER_SYMBOL = a.TICKER_SYMBOL 
	AND a.REPORT_DATE = b.REPORT_DATE
	join fund_asset_own c on c.TICKER_SYMBOL = b.TICKER_SYMBOL and c.REPORT_DATE = a.REPORT_DATE
WHERE
	IS_MAIN = 1 
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [134]:
df.groupby(by=[
    "REPORT_DATE",
	"LEVEL_1",
	"LEVEL_2",
	"LEVEL_3"]
).median().reset_index().to_excel("d:/指数1.xlsx")

基金指数

In [8]:
query_sql = """
select
date_format(TRADE_DT,"%Y%m%d") as TRADE_DT,
TICKER_SYMBOL,
S_DQ_CLOSE
from fund_index_eod
where 1 = 1 and TRADE_DT >= '20161230' and TICKER_SYMBOL in ("885062", "H11025", "931059")
"""

In [9]:
df = DB_CONN_JJTG_DATA.exec_query(query_sql)
df = df.pivot_table(index="TRADE_DT", columns="TICKER_SYMBOL", values="S_DQ_CLOSE").dropna()
df = df.pct_change().fillna(0)

In [5]:
df["平衡"] = df["885062"]*0.5 + df["H11025"]*.5
df["短债组合"] = df["885062"]*0.98 + df["H11025"]*.02


In [6]:
nav = (df["931059"] + 1).cumprod()
t = Performance(nav[nav.index>'20211230']).stats()
t

KeyError: '931059'

In [51]:
df.to_excel("D:/短债.xlsx")

In [47]:
df["平衡_log"] = np.log(df["平衡"] + 1)
df["短债组合_log"] = np.log(df["短债组合"] + 1)

In [50]:
df["平衡_log"].rolling(20).sum().dropna().describe()

count    1585.000000
mean        0.002426
std         0.001165
min        -0.002012
25%         0.001888
50%         0.002439
75%         0.002979
max         0.006222
Name: 平衡_log, dtype: float64

In [19]:
def get_fund_alpha(ticker_symbol, start_date, end_date, level_num=3):
    
    query_sql = f'''
    select 
    a.END_DATE, a.TICKER_SYMBOL, b.SEC_SHORT_NAME, 
    a.LEVEL_{level_num}, a.LOG_ALPHA_LEVEL_{level_num}
    from fund_derivatives_fund_log_alpha a
    join fund_info b on b.TICKER_SYMBOL = a.TICKER_SYMBOL
    where
    1=1
    and a.ticker_symbol = "{ticker_symbol}"
    and a.END_DATE between "{start_date}" and "{end_date}"
    and b.EXPIRE_DATE is NULL
    '''
    return DB_CONN_JJTG_DATA.exec_query(query_sql)

In [26]:
level_num=1
ticker_list = ["001406", "002010", "003592", "003638", "007100", "011034", "001442"]
ticker_list = [
    '000242',
'000739',
'002871',
'003501',
'004194',
'004641',
'005847',
'006377',
'007016',
'007126',
'007143',
'010338',
'010994',
'090007',
'110012',
'161611',
'166009',
'487016',
]
alpha_list = [
    get_fund_alpha(ticker, "20220808", "20230809", level_num)
    for ticker in ticker_list
]
alpha_df = pd.concat(alpha_list)

In [27]:
alpha_ret = alpha_df.pivot_table(
    index="END_DATE", columns="SEC_SHORT_NAME", values=f"LOG_ALPHA_LEVEL_{level_num}"
)

In [28]:
alpha_ret.corr()

SEC_SHORT_NAME,万家量化睿选A,中欧新动力混合(LOF)A,华夏智胜价值成长股票A,博时创新经济混合A,博道远航混合A,国投瑞银沪深300指数量化增强A,国投瑞银远见成长混合A,大成策略回报混合A,宏利睿智稳健混合A,富国沪港深业绩驱动混合型A,富国睿泽回报混合,工银灵活配置混合A,平安新鑫先锋混合A,广发趋势动力混合A,招商中证1000指数增强A,易方达科汇灵活配置混合,景顺长城策略精选灵活配置混合A,融通内需驱动混合A/B
SEC_SHORT_NAME,,,,,,,,,,,,,,,,,,
万家量化睿选A,1.000000,-0.105909,0.188411,0.176124,0.244320,-0.168438,-0.042677,0.010566,0.003338,-0.156743,-0.215510,0.171697,0.091376,0.146581,0.152410,-0.136762,-0.002774,0.074042
中欧新动力混合(LOF)A,-0.105909,1.000000,0.145379,0.029606,-0.060801,0.128527,0.197814,0.209425,0.123634,0.061937,0.242597,0.083092,-0.102671,0.074832,0.007729,0.129078,0.168445,0.051049
华夏智胜价值成长股票A,0.188411,0.145379,1.000000,0.292279,0.475700,0.012517,0.249826,0.460873,0.125448,-0.160730,0.007884,0.487414,0.136497,0.340386,0.584184,0.283717,0.323828,0.459732
博时创新经济混合A,0.176124,0.029606,0.292279,1.000000,0.325696,-0.478654,-0.204469,0.164870,0.201255,-0.367592,-0.258174,0.418554,0.275148,0.375467,0.396436,0.145722,0.213448,0.169646
博道远航混合A,0.244320,-0.060801,0.475700,0.325696,1.000000,-0.380430,-0.092247,-0.021022,0.255634,-0.330030,-0.265370,0.357995,0.380393,0.200307,0.578269,0.112300,0.129593,0.177854
国投瑞银沪深300指数量化增强A,-0.168438,0.128527,0.012517,-0.478654,-0.380430,1.000000,0.634644,0.291285,-0.294365,0.511495,0.602428,-0.445428,-0.485020,-0.452506,-0.413511,0.094984,-0.038294,-0.035313
国投瑞银远见成长混合A,-0.042677,0.197814,0.249826,-0.204469,-0.092247,0.634644,1.000000,0.461286,-0.174686,0.443918,0.611824,-0.101090,-0.395424,-0.282744,-0.204523,0.263110,-0.036302,0.131883
大成策略回报混合A,0.010566,0.209425,0.460873,0.164870,-0.021022,0.291285,0.461286,1.000000,-0.009236,0.113250,0.436360,0.316792,-0.325878,0.046275,-0.063618,0.449472,0.194686,0.368908
宏利睿智稳健混合A,0.003338,0.123634,0.125448,0.201255,0.255634,-0.294365,-0.174686,-0.009236,1.000000,-0.347037,-0.116031,0.277889,0.340947,0.181468,0.368199,0.050500,0.076160,-0.053572


In [70]:
alpha_ret

TICKER_SYMBOL,000242,000739,002871,003501,004194,004641,005847,006377,007016,007126,007143,010338,010994,090007,110012,161611,166009,487016
END_DATE,,,,,,,,,,,,,,,,,,
2022-01-04,0.111568,0.860537,1.211445,1.339279,0.525851,0.797591,-1.360638,0.527660,-0.083783,-0.746885,0.083308,1.861928,-0.906265,2.035142,1.900136,2.424422,0.514447,0.304518
2022-01-05,0.362993,0.192535,0.358406,0.521663,-0.772442,0.769449,0.456338,0.316404,1.337300,-0.380846,0.666298,2.011686,-1.169788,2.122107,0.700602,-0.223043,1.424268,0.407422
2022-01-06,1.622641,1.320420,0.504942,0.617116,1.364486,0.966106,-0.032202,1.774101,-0.821606,0.127477,-0.371821,0.011139,-0.478054,0.286520,0.008330,2.234369,0.059496,0.277313
2022-01-07,-0.082346,-0.950181,-0.023260,0.620369,-1.266203,-0.231125,-0.153011,0.003640,0.631724,-0.475689,0.512916,1.442848,-0.389465,1.476731,0.088947,-0.682258,1.175869,-0.072429
2022-01-10,-0.202233,-0.399271,0.404930,0.272123,0.390988,0.793000,-0.475930,-0.294056,0.141927,0.346048,-0.038555,0.148396,-0.606926,0.890252,0.446315,0.452963,0.412077,-0.302153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03,-0.147992,-0.553991,-0.122847,-0.081934,-0.120337,-0.216865,-0.172558,0.220840,0.107650,-0.161913,0.246332,0.013437,-0.772657,-0.133222,0.068684,-0.142518,-0.259607,-0.161827
2023-08-04,-0.459889,0.609628,-0.081288,-0.114125,0.486649,-0.098499,-0.192706,-0.184290,-0.374866,-0.231016,0.047755,-0.510038,0.311894,-0.356564,-0.275144,-0.878151,0.106179,-0.293700
2023-08-07,0.446376,-0.163431,-0.278302,0.055302,0.106511,0.092209,0.072723,-0.221889,0.133166,-0.087295,-0.063628,-0.165856,0.079530,-0.543098,-0.267520,0.437652,-0.133592,-0.187387


In [17]:
cundan = pd.read_excel("d:/存单基金.xlsx")
cundan = cundan.set_index("日期")["同业存单AAA"]

In [18]:
t = Performance(cundan[cundan.index>'20211230']).stats()
t

,0
起始日期,2021-12-31 00:00:00
结束日期,2023-08-10 00:00:00
累计收益率,0.041116
年化收益率,0.025503
年化波动率,0.002326
收益波动比,10.962469
最大回撤,0.001759
年化收益回撤比,14.502439
最大回撤日,20221117
最大回撤修复,18


In [16]:
cundan[cundan.index>'20211230']

,同业存单AAA
日期,
2021-12-31,127.93
2022-01-04,128.03
2022-01-05,128.04
2022-01-06,128.04
2022-01-07,128.06
...,...
2023-08-04,133.14
2023-08-07,133.16
2023-08-08,133.17
